In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc

In [ ]:
Celloc_results = pd.read_csv('./Celloc_output/DCIS1/mapping_results.csv',index_col=0)
print(Celloc_results)

In [ ]:
print(Celloc_results['CellType'].value_counts())

In [ ]:
st_adata=sc.read_h5ad("./data/DCIS1/spatial_DCIS1.h5ad")

sub_st_adata=st_adata[Celloc_results['Predict']]
sub_st_adata.obs['celltype']=list(Celloc_results['CellType'])
# sub_st_adata.obs['cell_type']=list(COMPASS_results['cell_type'])
sub_st_adata.obsm['spatial']=sub_st_adata.obsm['spatial'].astype(float)

coordinate_data=pd.DataFrame(sub_st_adata.obsm['spatial'],index=sub_st_adata.obs_names,columns=['x','y'])
def rand_jitter(arr,interval):
    return arr + np.random.uniform(-interval/3,interval/3,len(arr))
x_interval = np.median(np.unique(np.diff(np.sort(np.unique(coordinate_data['x'])))))*2
y_interval = np.median(np.unique(np.diff(np.sort(np.unique(coordinate_data['y'])))))*2
coordinate_data['x'] = rand_jitter(coordinate_data['x'].values,x_interval)
coordinate_data['y'] = rand_jitter(coordinate_data['y'].values,y_interval)
sub_st_adata.obsm['spatial']=np.array(coordinate_data[['x','y']])

sc.pl.spatial(sub_st_adata,spot_size=100,img_key='hires',color='celltype',alpha_img=0.6,save='Celloc_all_celltype.pdf',
              palette=['#014480','#52B7D7','#AEB02F','#645396','#CF5298','#94ABC9','#1F7539','#EBA272','#C2508F'])